#### A detailed documentation of the FXCM API is found under https://fxcm.github.io/rest-api-docs. To install the Python wrapper package fxcmpy, execute on the shell
#### pip install fxcmpy

In [ ]:
import fxcmpy
api = fxcmpy.fxcmpy(access_token=YOUR_FXCM_API_TOKEN, log_level='error')

In [ ]:
import time
import numpy as np
import pandas as pd 
import datetime as dt 
from pylab import mpl, plt

In [ ]:
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

####  Look at the available symbols (currency pairs) for which tick data is available.

In [ ]:
from fxcmpy import fxcmpy_tick_data_reader as tdr
print(tdr.get_available_symbols())

#### The following code retrieves one week worth of tick data for a single symbol. The resulting pandas DataFrame object has more than 1.5 million data rows.

#### This retrieves the data file, unpacks it and stores the raw data in a DataFrame object (as an attribute to the resulting object).

In [ ]:
start = dt.datetime(2018, 6, 25)
stop = dt.datetime(2018, 6, 30)
td = tdr('EURUSD', start, stop)
td.get_raw_data().info()

#### The td.get_raw_data() method returns the DataFrame object with the raw data, i.e. with the index values still being str objects.

#### The td.get_data() method returns a DataFrame object for which the index has been transformed to a DatetimeIndex.

In [ ]:
td.get_data().info()

In [ ]:
td.get_data().head()

#### Since the tick data is stored in a DataFrame object, it is straightforward to pick a sub-set of the data and to implement typical financial analytics tasks on it. Figure 14-1 shows a plot of the mid prices derived for the sub-set and a simple moving average (SMA).

In [ ]:
sub = td.get_data(start='2018-06-29 12:00:00',
                  end='2018-06-29 12:15:00')
sub.head()

In [ ]:
sub['Mid'] = sub.mean(axis=1)

In [ ]:
sub['SMA'] = sub['Mid'].rolling(1000).mean() 

In [ ]:
sub[['Mid', 'SMA']].plot(figsize=(10, 6), lw=0.75);

In [ ]:
from fxcmpy import fxcmpy_candles_data_reader as cdr
print(cdr.get_available_symbols())

In [ ]:
start = dt.datetime(2018, 5, 1)
stop = dt.datetime(2018, 6, 30)
period = 'H1'
candles = cdr('EURUSD', start, stop, period)
data = candles.get_data()
data = candles.get_data()

#### Open, high, low, close values for the bid prices.

In [ ]:
data[data.columns[:4]].tail()

#### Open, high, low, close values for the ask prices.

In [ ]:
data[data.columns[4:]].tail()

In [ ]:
data['MidClose'] = data[['BidClose', 'AskClose']].mean(axis=1)

In [ ]:
data['SMA1'] = data['MidClose'].rolling(30).mean()
data['SMA2'] = data['MidClose'].rolling(100).mean()
data[['MidClose', 'SMA1', 'SMA2']].plot(figsize=(10, 6));

### Working with the API
#### While the previous sections retrieve historical tick data and candles data pre-packaged from FXCM servers, this subsequent section shows how to retrieve historical data via the API. However, a connection object to the FXCM API is needed. Therefore, first the import of the fxcmpy package, the connection to the API (based on the unique API token) and a look at the available instruments.

In [ ]:
import fxcmpy
fxcmpy.__version__

In [ ]:
api = fxcmpy.fxcmpy(config_file='../fxcm.cfg')

In [ ]:
instruments = api.get_instruments()
print(instruments)

### Retrieving Historical Data
#### Once connected, data retrieval for specific time intervals is accomplished via a single method call. When using the get_candles() method, the parameter period can be one of m1, m5, m15, m30, H1, H2, H3, H4, H6, H8, D1, W1 or M1. Figure 14-3 shows one minute bar ask close prices for the EUR/USD instrument (currency pair).

#### Retrieves the 10 most recent end-of-day prices.

In [ ]:
candles[candles.columns[4:]]

#### Retrieves end-of-day prices for a whole year.

In [ ]:
start = dt.datetime(2017, 1, 1)
end = dt.datetime(2018, 1, 1)
candles = api.get_candles('EUR/GBP', period='D1',
                          start=start, stop=end)

In [ ]:
candles.info()

#### Retrieves the most recent one minute bar prices available.

In [ ]:
candles = api.get_candles('EUR/USD', period='m1', number=250)
candles['askclose'].plot(figsize=(10, 6))

### Retrieving Streaming 
#### DataWhile historical data is important to, for example, backtest algorithmic trading strategies, continuous access to real-time or streaming data (during trading hours) is required to deploy and automate algorithmic trading strategies. The FXCM API allows for the subscription to real-time data streams for all instruments. The fxcmpy wrapper package supports this functionality, among others, in that it allows to provide user-defined functions (so called callback functions) to process the real-time data stream.The following code presents such a simple callback function — it only prints out selected elements of the data set retrieved — and uses it to process data retrieved in real-time, after a subscription for the desired instrument (here EUR/USD).

#### The callback function that prints out certain elements of the retrieved data set.

In [ ]:
def output(data, dataframe):
    print('%3d | %s | %s | %6.5f, %6.5f'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1]))

#### The subscription to a specific real-time data stream; data is processed asynchronously as long as there is no “unsubscribe” event.

In [ ]:
api.subscribe_market_data('EUR/USD', (output,))

#### During the subscription, the .get_last_price() method returns the last available data set.

In [ ]:
api.get_last_price('EUR/USD')

#### This unsubscribes from the real-time data stream.

In [ ]:
api.unsubscribe_market_data('EUR/USD')

#### Shows the open positions for the connected (default) account.

In [ ]:
api.get_open_positions()

#### Opens a position of 100,000 in the EUR/USD currency pair

In [ ]:
order = api.create_market_buy_order('EUR/USD', 100)
sel = ['tradeId', 'amountK', 'currency',
       'grossPL', 'isBuy']
api.get_open_positions()[sel]

In [ ]:
order = api.create_market_buy_order('EUR/GBP', 50)

In [ ]:
api.get_open_positions()[sel]

#### While the create_market_buy_order() open or increases positions, the create_market_sell_order() allows to close or decrease positions. There are also more general methods that allow the closing out of position as the code below illustrates.

In [ ]:
order = api.create_market_sell_order('EUR/USD', 25)
order = api.create_market_buy_order('EUR/GBP', 50)

In [ ]:
api.get_open_positions()[sel]

In [ ]:
api.close_all_for_symbol('EUR/GBP')

In [ ]:
api.get_open_positions()[sel]

In [ ]:
api.close_all()

In [ ]:
api.get_open_positions()

### Account Information
#### Beyond, for example, open positions, the FXCM API allows to retrieve more general account information as well. For example, one can look up the default account (if there are multiple accounts) or an overview equity and margin situation.

#### Shows the default accountId value.

In [ ]:
api.get_default_account()

#### Shows for all accounts the financial situation and some parameters.

In [ ]:
api.get_accounts().T